# Library importation

In [1]:
import numpy as np
import pandas as pd

# Code for countries of OECD

In [2]:
oecd_countries = {'AUS': 'Australia', 'AUT': 'Austria', 'BEL': 'Belgium', 'CAN': 'Canada', 'CHL': 'Chile', 'COL': 'Colombia', 'CZE': 'Czech Republic', 'DNK': 'Denmark', 'EST': 'Estonia', 'FIN': 'Finland', 'FRA': 'France', 'DEU': 'Germany', 'GRC': 'Greece', 'HUN': 'Hungary', 'ISL': 'Iceland', 'IRL': 'Ireland', 'ISR': 'Israel', 'ITA': 'Italy', 'JPN': 'Japan', 'KOR': 'Korea', 'LVA': 'Latvia', 'LTU': 'Lithuania', 'LUX': 'Luxembourg', 'MEX': 'Mexico', 'NLD': 'Netherlands', 'NZL': 'New Zealand', 'NOR': 'Norway', 'POL': 'Poland', 'PRT': 'Portugal', 'SVK': 'Slovakia', 'SVN': 'Slovenia', 'ESP': 'Spain', 'SWE': 'Sweden', 'CHE': 'Switzerland', 'TUR': 'Turkey', 'GBR': 'United Kingdom', 'USA': 'United States'}

# Data importation

## Starting with the NEET

We select the neet corresponding from 15 to 29 yo and from OECD countries only and selecting all the years

In [3]:
df = pd.read_csv('./data/education_system/NEET_men_women_altogether.csv')
df = df[(df.SUBJECT=='15_29')].drop(['INDICATOR','SUBJECT','MEASURE','FREQUENCY','Flag Codes'],axis=1)
df = df[(df.LOCATION).isin(oecd_countries.keys())]
df = df.reset_index(drop=True)

In [4]:
df.columns = ['Country','Time','NEET']

## Looking at Expenditure on LMP

### Preparing the dataframe

In [5]:
exp_lmp = pd.read_csv('./data/labour_market/public_exp_LMP.csv')
exp_lmp = exp_lmp[['LFS_COUNTRY','PROG','Time','Value']]
exp_lmp = exp_lmp[exp_lmp.PROG == 100].drop('PROG',axis=1).reset_index(drop=True)

In [6]:
exp_lmp.columns=['Country', 'Time', 'Exp_LMP']

In [7]:
type(exp_lmp['Time'])

pandas.core.series.Series

### Adding the values to the DF

In [8]:
list_value=[]
for el in df.itertuples():
    country,time = el[1],el[2]
    tmp = exp_lmp[(exp_lmp.Country==country)&(exp_lmp.Time==time)]
    if len(tmp.index)==0:
        list_value.append(None)
    else:
        list_value.append(tmp.Exp_LMP.values[0])
df['Exp_LMP']= list_value
df['Exp_LMP_2']=df['Exp_LMP']**2

In [9]:
df

,Country,Time,NEET,Exp_LMP,Exp_LMP_2
0,AUS,1997,16.034595,NaN,NaN
1,AUS,1998,14.880802,NaN,NaN
2,AUS,1999,13.637552,NaN,NaN
3,AUS,2000,13.192960,NaN,NaN
4,AUS,2001,12.979000,NaN,NaN
...,...,...,...,...,...
691,LTU,2015,13.743647,0.53,0.2809
692,LTU,2016,11.403278,0.51,0.2601
693,LTU,2017,11.220660,0.54,0.2916
694,LTU,2018,10.495108,0.66,0.4356


## Looking at STR

### Preparing the dataset

In [10]:
STR = pd.read_csv('./data/education_system/student_teacher_ratio.csv')
STR = STR[['COUNTRY','ISC11_LEVEL_CAT','Year','Value']]
STR_L1 = STR[STR.ISC11_LEVEL_CAT == 'L1'].drop('ISC11_LEVEL_CAT',axis=1).reset_index(drop=True)

In [11]:
STR_L1.columns = ['Country', 'Time', 'STR']

In [12]:
STR_L1

,Country,Time,STR
0,AUS,2013,15.615
1,AUS,2014,15.612
2,AUS,2015,15.433
3,AUS,2016,15.168
4,AUS,2017,15.124
...,...,...,...
228,SVN,2014,15.862
229,SVN,2015,15.888
230,SVN,2016,14.282
231,SVN,2017,14.450


### Adding the values to the DF

In [13]:
list_value=[]
for el in df.itertuples():
    country,time = el[1],el[2]
    tmp = STR_L1[(STR_L1.Country==country)&(STR_L1.Time==time)]
    if len(tmp.index)==0:
        list_value.append(None)
    else:
        list_value.append(tmp.STR.values[0])
df['STR']= list_value
df['STR_2']=df['STR']**2
df['STR_3']=df['STR']**3

## Looking at economic features

### GDP and LogGDP

In [14]:
gdp_per_person = pd.read_csv('./data/economic_features/GDP_per_capita.csv', sep=';')
gdp_per_person_oecd = pd.DataFrame(columns=gdp_per_person.columns)
for code in oecd_countries.keys():
    gdp_per_person_oecd = gdp_per_person_oecd.append(gdp_per_person[gdp_per_person['Country Code']==code])
gdp_2015_oecd = gdp_per_person_oecd[['Country Code', '2015']]
gdp_2017_oecd = gdp_per_person_oecd[['Country Code', '2017']]
gdp_1960_2019_oecd_full = gdp_per_person_oecd.drop(['Country Name', 'Indicator Name', 'Indicator Code'], axis=1).reset_index(drop=True)

gdp_1960_2019_oecd = pd.DataFrame(columns=['Country', 'Time', 'GDP'])
for i in range(len(gdp_1960_2019_oecd_full)):
    country = gdp_1960_2019_oecd_full.iloc[i][0]
    for j in range(1,len(gdp_1960_2019_oecd_full.columns)):
        year = gdp_1960_2019_oecd_full.columns[j]
        gdp = gdp_1960_2019_oecd_full.iloc[i][j]
        gdp_1960_2019_oecd = gdp_1960_2019_oecd.append({'Country': country, 'Time': int(year), 'GDP': gdp}, ignore_index=True)
    gdp_1960_2019_oecd = gdp_1960_2019_oecd.dropna()

In [15]:
list_value=[]
for el in df.itertuples():
    country,time = el[1],el[2]
    tmp = gdp_1960_2019_oecd[(gdp_1960_2019_oecd.Country==country)&(gdp_1960_2019_oecd.Time==time)]
    if len(tmp.index)==0:
        list_value.append(None)
    else:
        list_value.append(tmp.GDP.values[0])
df['GDP']= list_value
df['LogGDP']=np.log(df['GDP'])

### CPI

In [16]:
cpi_2000_2019 = pd.read_csv('./data/economic_features/CPI_2000_2019.csv')
cpi_2000_2019 = cpi_2000_2019[['LOCATION', 'TIME', 'Value']]
cpi_2000_2019.columns = ['Country', 'Time', 'CPI']

In [17]:
list_value=[]
for el in df.itertuples():
    country,time = el[1],el[2]
    tmp = cpi_2000_2019[(cpi_2000_2019.Country==country)&(cpi_2000_2019.Time==time)]
    if len(tmp.index)==0:
        list_value.append(None)
    else:
        list_value.append(tmp.CPI.values[0])
df['CPI']= list_value

### DEBT

In [18]:
debt_2000_2019 = pd.read_csv('./data/economic_features/DEBT_2000_2019.csv')
debt_2000_2019 = debt_2000_2019[['LOCATION', 'TIME', 'Value']]
debt_2000_2019.columns = ['Country', 'Time', 'DEBT']

In [19]:
list_value=[]
for el in df.itertuples():
    country,time = el[1],el[2]
    tmp = debt_2000_2019[(debt_2000_2019.Country==country)&(debt_2000_2019.Time==time)]
    if len(tmp.index)==0:
        list_value.append(None)
    else:
        list_value.append(tmp.DEBT.values[0])
df['DEBT']= list_value

## Looking at education features

### Years of schooling

In [20]:
years_schooling = pd.read_csv('./data/education_system/mean-years-of-schooling-world.csv')
years_schooling=years_schooling.drop('Entity',axis=1)
years_schooling.columns=['Country', 'Time', 'Years_schooling']

In [21]:
list_value=[]
for el in df.itertuples():
    country,time = el[1],el[2]
    tmp = years_schooling[(years_schooling.Country==country)&(years_schooling.Time==time)]
    if len(tmp.index)==0:
        list_value.append(None)
    else:
        list_value.append(tmp.Years_schooling.values[0])
df['Years_schooling']= list_value

### Average class size

In [22]:
avg_class_size = pd.read_csv('./data/education_system/Avg_class_size_primary_education.csv')
avg_class_size = avg_class_size[(avg_class_size.ISC11_LEVEL_CAT=='L1')&(avg_class_size.REF_SECTOR=='INST_T')&(avg_class_size.INDICATOR=='PERS_AVG_CLASS')]
avg_class_size = avg_class_size[['COUNTRY','Year','Value']]
avg_class_size.columns = ['Country', 'Time', 'Avg_class_size']

In [23]:
list_value=[]
for el in df.itertuples():
    country,time = el[1],el[2]
    tmp = avg_class_size[(avg_class_size.Country==country)&(avg_class_size.Time==time)]
    if len(tmp.index)==0:
        list_value.append(None)
    else:
        list_value.append(tmp.Avg_class_size.values[0])
df['Avg_class_size']= list_value

### Expenditure on education

In [24]:
spendings_educ = pd.read_csv('./data/education_system/spendings_in_education.csv')
spendings_educ = spendings_educ[(spendings_educ.ISC11=='L1')& (spendings_educ.EXPENDITURE_TYPE=='T')]
spendings_educ = spendings_educ[['COUNTRY', 'YEAR','Value']]
spendings_educ.columns = ['Country', 'Time', 'Exp_educ']

In [25]:
spendings_educ

,Country,Time,Exp_educ
98,AUS,1995,NaN
99,AUS,2000,4549.9614
100,AUS,2005,6003.6421
101,AUS,2008,7103.9819
102,AUS,2009,8588.7402
...,...,...,...
14885,KOR,2014,NaN
14886,KOR,2015,NaN
14887,KOR,2016,11074.5190
14888,KOR,2017,11701.7900


In [26]:
list_value=[]
for el in df.itertuples():
    country,time = el[1],el[2]
    tmp = spendings_educ[(spendings_educ.Country==country)&(spendings_educ.Time==time)]
    if len(tmp.index)==0:
        list_value.append(None)
    else:
        list_value.append(tmp.Exp_educ.values[0])
df['Exp_educ']= list_value

df['LogExp_educ']= np.log(df['Exp_educ'],where=~np.isnan(np.array(df['Exp_educ'])))

## Labour indicators

### Protection of workers

In [27]:
strictness = pd.read_csv('./data/labour_market/strictness of employment protection from 1998 to 2018.csv')
strictness = strictness[(strictness.SERIES == 'EPRC_V2')]
strictness = strictness[['COUNTRY','TIME','Value']]
strictness.columns = ['Country', 'Time', 'Strictness_of_workers']

In [28]:
list_value=[]
for el in df.itertuples():
    country,time = el[1],el[2]
    tmp = strictness[(strictness.Country==country)&(strictness.Time==time)]
    if len(tmp.index)==0:
        list_value.append(None)
    else:
        list_value.append(tmp.Strictness_of_workers.values[0])
df['Strictness_of_workers']= list_value

### Percentage of Part time employement

In [29]:
ft_pt_employ = pd.read_csv('./data/labour_market/percentage of full time part time employment.csv')
ft_pt_employ = ft_pt_employ[(ft_pt_employ.AGE==1524)&(ft_pt_employ.EMPSTAT=='TE')&(ft_pt_employ.SEX=='MW')]
ft_pt_employ = ft_pt_employ[['COUNTRY','SERIES','Time','Value']]
ft_pt_employ.columns = ['Country','Series','Time','Value']

pt_employ = ft_pt_employ[ft_pt_employ.Series=='PT'].drop('Series',axis=1)

In [30]:
list_value = []
for el in df.itertuples():
    country, time = el[1], el[2]
    tmp = pt_employ[(pt_employ.Country == country) & (pt_employ.Time == time)]
    if len(tmp.index) == 0:
        list_value.append(None)
    else:
        list_value.append(tmp.Value.values[0])
df['PT_employ'] = list_value

### Average duration of unemployement

In [31]:
avg_duration_unemployment = pd.read_csv('./data/labour_market/average duration of unemployment.csv')
avg_duration_unemployment = avg_duration_unemployment[(avg_duration_unemployment.SEX=='MW')&(avg_duration_unemployment.AGE==1524)]
avg_duration_unemployment = avg_duration_unemployment[['COUNTRY','TIME','Value']]
avg_duration_unemployment.columns = ['Country', 'Time','Avg_dur_unemployment']
len(avg_duration_unemployment)

184

In [32]:
list_value = []
for el in df.itertuples():
    country, time = el[1], el[2]
    tmp = avg_duration_unemployment[(avg_duration_unemployment.Country == country) & (avg_duration_unemployment.Time == time)]
    if len(tmp.index) == 0:
        list_value.append(None)
    else:
        list_value.append(tmp.Avg_dur_unemployment.values[0])
df['Avg_dur_unemployment'] = list_value

In [38]:
df.to_csv('./data/panel_data/Full_DB.csv',index=False)